In [125]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hw-texts/baseline_submit.csv
/kaggle/input/hw-texts/train.csv
/kaggle/input/hw-texts/test.csv


In [126]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, roc_auc_score, classification_report, precision_score, recall_score, make_scorer
import joblib
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from pymorphy2.tokenizers import simple_word_tokenize

In [127]:
! pip install pymorphy2[fast] rnnmorph pymystem3 nltk razdel

In [128]:
train = pd.read_csv('/kaggle/input/hw-texts/train.csv', sep=';')
test = pd.read_csv('/kaggle/input/hw-texts/test.csv', sep=';')

regex = re.compile(r'(\W)\1+')
for i in range(len(train['text'])):
    text = train['text'][i].lower()
    newtext1 = regex.sub(r'\1', text)
    newtext2 = re.sub('\s+', ' ', newtext1)
    train['text'][i] = simple_word_tokenize(newtext2)
        
    

for i in range(len(test['text'])):
    text = test['text'][i].lower()
    newtext1 = regex.sub(r'\1', text)
    newtext2 = re.sub('\s+', ' ', newtext1)
    test['text'][i] = simple_word_tokenize(newtext2)

In [129]:
%matplotlib inline

import tqdm
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


def evaluate_vectorizer(vectorizer):
    train_vectors = vectorizer.fit_transform(train['text'])
    test_vectors = vectorizer.transform(test['text'])
    
    clf = LinearSVC(random_state=42)
    
    clf.fit(train_vectors, train['label'])
    
    predictions = clf.predict(test_vectors)
    if (test['label'] is not None):
        print(classification_report(test['label'], predictions))
        return predictions

In [130]:
evaluate_vectorizer(CountVectorizer(min_df=2))

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


KeyError: 'label'

In [ ]:
evaluate_vectorizer(TfidfVectorizer(min_df=2))

In [ ]:
def tokenize_with_razdel(text):
    tokens = [token.text for token in razdel.tokenize(text)]
    
    return tokens

In [ ]:
evaluate_vectorizer(TfidfVectorizer(min_df=2, tokenizer=tokenize_with_razdel));

In [ ]:
tfidf_vectorizer = TfidfVectorizer(
    min_df=2, 
    tokenizer=lambda text: lemmatize_with_pymorphy(tokenize_with_razdel(text)),
)

predictions=evaluate_vectorizer(tfidf_vectorizer)